In [25]:
from PIL import Image,ImageFilter,ImageOps
import numpy as np
from os import listdir
import h5py
from google.colab import drive
import cv2
import random

In [26]:
# Load the Drive helper and mount
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#navigating to the directory
%cd drive/My\ Drive/data/test

[Errno 2] No such file or directory: 'drive/My Drive/data/test'
/content/drive/My Drive/data/test


In [53]:
%cd train/

/content/drive/My Drive/data/train


In [3]:
!ls

paper  rock  scissors  test_data_32.h5	test_data_64.h5


In [4]:
#cosnt 
SIZE_IMAGE = 32

In [5]:
#convert array np to image
def convert_np_array_to_image(im_np):
    return Image.fromarray(np.uint8(im_np)).convert("RGB")

In [37]:
#loading class 
def load_class(dir,imagens):
    for filename in listdir(dir):
        path = dir + filename
        try:
            imagens.extend(open_image_to_np_array(True, path))
        except:
            print("Erro ao ler imagem {}".format(path))

In [44]:
#rotate images
def rotate_image(image):
    im_45 = rotate_in_angle(np.asarray(image),10)
    im_90 = rotate_in_angle(np.asarray(image),20)
    im_135 = rotate_in_angle(np.asarray(image),-10)
    im_180 = rotate_in_angle(np.asarray(image),-20)
    return [np.asarray(image),im_45,im_90,im_135,im_180]

In [45]:
#Girando a imagem toda
def rotate_in_angle(img_a,angle):
    rows, cols = img_a.shape[0:2]
    rot = cv2.getRotationMatrix2D(((cols - 1)/2.0, (rows - 1)/2.0),angle,1)
    dst = cv2.warpAffine(img_a, rot, (cols, rows), borderValue = [255, 255, 255])
    return dst

In [46]:
#blue effect
def blur_filter(image):
    return image.filter(ImageFilter.GaussianBlur(3))

In [19]:
def open_image_to_np_array(rotate,dir):
    image = Image.open(dir)
    image = image.convert("RGB")
    image = image.resize((SIZE_IMAGE,SIZE_IMAGE))
    if rotate:
        return rotate_image(image)
    else:
        return [np.asarray(image)]

In [20]:
def zoom_in(image,zoom_in_value):
    return image.crop(((SIZE_IMAGE/2)-1 - SIZE_IMAGE/(zoom_in_value+1), (SIZE_IMAGE/2)-1 - SIZE_IMAGE/(zoom_in_value+1), (SIZE_IMAGE/2)-1 + SIZE_IMAGE/(zoom_in_value+1), (SIZE_IMAGE/2)-1 + SIZE_IMAGE/(zoom_in_value+1))).resize((SIZE_IMAGE,SIZE_IMAGE))

In [21]:
def zoom_out(image,zoom_out_value_percent):
    return ImageOps.expand(image, border = int((SIZE_IMAGE/100) * zoom_out_value_percent), fill = 'white').resize((SIZE_IMAGE,SIZE_IMAGE))

In [22]:
def apply_zoom(list_of_images):
    images_to_zoom = np.random.choice(len(list_of_images),size=int(len(list_of_images)/100*20))
    for index_image in images_to_zoom:
        if index_image > len(images_to_zoom)/2:
            list_of_images.append(np.asarray(zoom_in(convert_np_array_to_image(list_of_images[index_image]),random.random()+1.5)))
        else:
            list_of_images.append(np.asarray(zoom_out(convert_np_array_to_image(list_of_images[index_image]),random.randint(10,50))))

In [54]:
#export dataset to h5 file
images_paper = list()
images_rock = list()
images_scissors = list()
load_class("paper/",images_paper)
load_class("rock/",images_rock)
load_class("scissors/",images_scissors)
apply_zoom(images_paper)
apply_zoom(images_rock)
apply_zoom(images_scissors)

In [55]:
labels = len(images_paper) * [0] +  len(images_rock) * [1] + len(images_scissors) * [2]
images = images_paper + images_rock + images_scissors
images = np.array(images)
labels = np.array(labels)

In [56]:

dataset_file = h5py.File("train_data_no_rotate_32.h5", "w")
dataset = dataset_file.create_dataset("train_set_x", data=images)
dataset = dataset_file.create_dataset("train_set_y", data=labels)
dataset = dataset_file.create_dataset("list_classes", data=np.array(["paper","rock","scissors"], dtype='|S7'))
dataset_file.flush()
dataset_file.close()

In [57]:
images.shape

(15120, 32, 32, 3)